In [1]:
import RevPrep

In [2]:
import pandas as pd

In [3]:
import pickle as pkl
import glob
import numpy as np

In [4]:
#Using Bookbrowse
file = glob.glob("*.csv")[0]
df = pd.read_csv(file, delimiter=";", encoding="latin-1")
summaries = list(df["Summary"])

IndexError: list index out of range

In [5]:
#Using Goodreads
summaries = pkl.load(open("window_ds_big.pkl", "rb"))

In [6]:
from RevPrep.Helpers.helpers import text_to_sentences
sen_counts = [len(text_to_sentences(i)) for i in summaries]

In [7]:
from tensorflow import keras as k
import numpy as np
tokenizertf = k.preprocessing.text.Tokenizer()
tokenizertf.fit_on_texts(summaries)

In [8]:
def is_good(model_name, sums, tokenizer, starti=6, rangei=16, inc=2, overlap_inc=1, max_overlap=5):
    from RevPrep.Generation.generate.base import generate 
    from RevPrep.Evaluations.scoring import analyse_sentiment, cal_generation_quality
    from RevPrep.Generation.generate.base import get_model_tokenizer, del_model_tokenizer
    from RevPrep.plotting import plot
    from tqdm.notebook import tqdm
    import numpy as np
    good_configs = []
    bad_configs = []
    model,tokenizer = get_model_tokenizer(model_name) 
    for i in tqdm(range(starti,rangei,inc), position=0, desc=f"Up to {rangei} windows, start: {starti}"):
        for i2 in tqdm(range(0,int(np.min([i/2, max_overlap])),overlap_inc), desc=f"Up to {int(np.min([i/2, max_overlap]))} overlap, start: 0", position=0, leave=False):
            good_method = False
            results_window = generate(model_name, sums, model, tokenizer, True, i,i2,True)
            c_rate = results_window['compression_rate']
            gen_qual = cal_generation_quality(results_window["generated summaries"], results_window["source_texts"], tokenizertf)
            del results_window
        
            if(gen_qual["mean"] > 0.40 and c_rate > 0.40 and c_rate < 0.55):
                good_method = True
        
            if good_method:
                good_configs.append({"model_name": model_name, "window_size": i, "overlap": i2, "compression_rate": c_rate, "gen_quality mean": gen_qual["mean"], "gen_quality std dev": gen_qual["std dev"], "example_gen": gen_qual["example_gen"], "example_src": gen_qual["example_source"]})
            else:
                bad_configs.append({"model_name": model_name, "window_size": i, "overlap": i2, "compression_rate": c_rate, "gen_quality mean": gen_qual["mean"], "gen_quality std dev": gen_qual["std dev"], "example_gen": gen_qual["example_gen"], "example_src": gen_qual["example_source"]})
    del_model_tokenizer(model, tokenizer)
    return good_configs, bad_configs

In [9]:
good_configs = {}
bad_configs = {}

### For Pegasus Wikihow 

In [ ]:
model_name = "google/pegasus-wikihow"
good_configs[model_name], bad_configs[model_name] = is_good(model_name, summaries[:4], tokenizertf, 80, 300, 20, 10, 60)

if len(good_configs[model_name]) > 0:
    print(f"good configs: {len(good_configs[model_name])}, best config: {np.argmax([i['gen_quality mean'] for i in good_configs[model_name]])}")
else:
    best_index = np.argmax([i['gen_quality mean'] for i in bad_configs[model_name]])
    print(f"No good config found. best config so far reached: {best_index} with mean: {bad_configs[model_name][best_index]['gen_quality mean']} and comp: {bad_configs[model_name][best_index]['compression_rate']}")
    
pkl.dump(good_configs, open("./good_configs.pkl", "wb"))
pkl.dump(bad_configs, open("./bad_configs.pkl", "wb"))

PyTorch version 1.6.0 available.
TensorFlow version 2.3.1 available.


In [11]:
bad_configs

{'google/pegasus-wikihow': [{'model_name': 'google/pegasus-wikihow',
   'window_size': 80,
   'overlap': 0,
   'compression_rate': 0.18238946805478945,
   'gen_quality mean': 0.08528997980266001,
   'gen_quality std dev': 0.030578153052295727,
   'example_gen': 'What I Liked: This is my first Wendy Higgins book, so I cant this one to her Sweet series, or her selfpublished novel.<n>What I Liked: This is my first Wendy Higgins book, so I cant this one to her Sweet series, or her selfpublished novel.<n>What I Liked: This is my first Wendy Higgins book, so I cant this one to her Sweet series, or her selfpublished novel.<n>What I Liked: This is my first Wendy Higgins book, so I cant this one to her Sweet series, or her selfpublished novel.<n>What I Liked: This is my first Wendy Higgins book, so I cant this one to her Sweet series, or her selfpublished novel.',
   'example_src': 'Such a great book... SO good... until the ending. I dont know if I want to be lenient because I know there will b

### For Pegasus: reddit_tifu Window Size 1 - 7 

In [10]:
good_configs = pkl.load(open("./good_configs.pkl","rb"))
bad_configs = pkl.load(open("./bad_configs.pkl","rb"))

model_name = "google/pegasus-reddit_tifu"
good_configs[model_name], bad_configs[model_name] = is_good(model_name, summaries[:4], tokenizertf, 18)

if len(good_configs[model_name]) > 0:
    print(f"good configs: {len(good_configs[model_name])}, best config: {np.argmax([i['gen_quality mean'] for i in good_configs[model_name]])}")
else:
    best_index = np.argmax([i['gen_quality mean'] for i in bad_configs[model_name]])
    print(f"No good config found. best config so far reached: {best_index} with mean: {bad_configs[model_name][best_index]['gen_quality mean']} and comp: {bad_configs[model_name][best_index]['compression_rate']}")

pkl.dump(good_configs, open("./good_configs.pkl", "wb"))
pkl.dump(bad_configs, open("./bad_configs.pkl", "wb"))

PyTorch version 1.6.0 available.
TensorFlow version 2.3.1 available.



No good config found. best config so far reached: 11 with mean: 0.3617069420205956 and comp: 0.5428049679722788


### For Pegasus cnn-dailymail:  Window Size 1-7

In [11]:
model_name = "google/pegasus-cnn_dailymail"
good_configs = pkl.load(open("./good_configs.pkl","rb"))
bad_configs = pkl.load(open("./bad_configs.pkl","rb"))

good_configs[model_name], bad_configs[model_name] = is_good(model_name, summaries[:4], tokenizertf, 18)

if len(good_configs[model_name]) > 0:
    print(f"good configs: {len(good_configs[model_name])}, best config: {np.argmax([i['gen_quality mean'] for i in good_configs[model_name]])}")
else:
    best_index = np.argmax([i['gen_quality mean'] for i in bad_configs[model_name]])
    print(f"No good config found. best config so far reached: {best_index} with mean: {bad_configs[model_name][best_index]['gen_quality mean']} and comp: {bad_configs[model_name][best_index]['compression_rate']}")
    
pkl.dump(good_configs, open("./good_configs.pkl", "wb"))
pkl.dump(bad_configs, open("./bad_configs.pkl", "wb"))

KeyboardInterrupt: 

### For Pegasus xsum:  Window Size 1-7

In [ ]:
model_name = "google/pegasus-xsum"
good_configs = pkl.load(open("./good_configs.pkl","rb"))
bad_configs = pkl.load(open("./bad_configs.pkl","rb"))

good_configs[model_name], bad_configs[model_name] = is_good(model_name, summaries[:4], tokenizertf, 18)

if len(good_configs[model_name]) > 0:
    print(f"good configs: {len(good_configs[model_name])}, best config: {np.argmax([i['gen_quality mean'] for i in good_configs[model_name]])}")
else:
    best_index = np.argmax([i['gen_quality mean'] for i in bad_configs[model_name]])
    print(f"No good config found. best config so far reached: {best_index} with mean: {bad_configs[model_name][best_index]['gen_quality mean']} and comp: {bad_configs[model_name][best_index]['compression_rate']}")
    
pkl.dump(good_configs, open("./good_configs.pkl", "wb"))
pkl.dump(bad_configs, open("./bad_configs.pkl", "wb"))

### For Pegasus Large: Window Size 1-7 

In [ ]:
model_name = "google/pegasus-large"
good_configs = pkl.load(open("./good_configs.pkl","rb"))
bad_configs = pkl.load(open("./bad_configs.pkl","rb"))

good_configs[model_name], bad_configs[model_name] = is_good(model_name, summaries[:4], tokenizertf, 18)

if len(good_configs[model_name]) > 0:
    print(f"good configs: {len(good_configs[model_name])}, best config: {np.argmax([i['gen_quality mean'] for i in good_configs[model_name]])}")
else:
    best_index = np.argmax([i['gen_quality mean'] for i in bad_configs[model_name]])
    print(f"No good config found. best config so far reached: {best_index} with mean: {bad_configs[model_name][best_index]['gen_quality mean']} and comp: {bad_configs[model_name][best_index]['compression_rate']}")

pkl.dump(good_configs, open("./good_configs.pkl", "wb"))
pkl.dump(bad_configs, open("./bad_configs.pkl", "wb"))

### For Pegasus Newsroom

In [ ]:
model_name = "google/pegasus-newsroom"

good_configs = pkl.load(open("./good_configs.pkl","rb"))
bad_configs = pkl.load(open("./bad_configs.pkl","rb"))
good_configs[model_name], bad_configs[model_name] = is_good(model_name, summaries[:4], tokenizertf, 18)

if len(good_configs[model_name]) > 0:
    print(f"good configs: {len(good_configs[model_name])}, best config: {np.argmax([i['gen_quality mean'] for i in good_configs[model_name]])}")
else:
    best_index = np.argmax([i['gen_quality mean'] for i in bad_configs[model_name]])
    print(f"No good config found. best config so far reached: {best_index} with mean: {bad_configs[model_name][best_index]['gen_quality mean']} and comp: {bad_configs[model_name][best_index]['compression_rate']}")

pkl.dump(good_configs, open("./good_configs.pkl", "wb"))
pkl.dump(bad_configs, open("./bad_configs.pkl", "wb"))

### For Pegasus Multinews 

In [ ]:
model_name = "google/pegasus-multi_news"

good_configs = pkl.load(open("./good_configs.pkl","rb"))
bad_configs = pkl.load(open("./bad_configs.pkl","rb"))
good_configs[model_name], bad_configs[model_name] = is_good(model_name, summaries[:4], tokenizertf, 18)

if len(good_configs[model_name]) > 0:
    print(f"good configs: {len(good_configs[model_name])}, best config: {np.argmax([i['gen_quality mean'] for i in good_configs[model_name]])}")
else:
    best_index = np.argmax([i['gen_quality mean'] for i in bad_configs[model_name]])
    print(f"No good config found. best config so far reached: {best_index} with mean: {bad_configs[model_name][best_index]['gen_quality mean']} and comp: {bad_configs[model_name][best_index]['compression_rate']}")
    
pkl.dump(good_configs, open("./good_configs.pkl", "wb"))
pkl.dump(bad_configs, open("./bad_configs.pkl", "wb"))

### For Pegasus Gigaword 

In [ ]:
model_name = "google/pegasus-gigaword"
good_configs = pkl.load(open("./good_configs.pkl","rb"))
bad_configs = pkl.load(open("./bad_configs.pkl","rb"))
good_configs[model_name], bad_configs[model_name] = is_good(model_name, summaries[:4], tokenizertf, 18)

if len(good_configs[model_name]) > 0:
    print(f"good configs: {len(good_configs[model_name])}, best config: {np.argmax([i['gen_quality mean'] for i in good_configs[model_name]])}")
else:
    best_index = np.argmax([i['gen_quality mean'] for i in bad_configs[model_name]])
    print(f"No good config found. best config so far reached: {best_index} with mean: {bad_configs[model_name][best_index]['gen_quality mean']} and comp: {bad_configs[model_name][best_index]['compression_rate']}")

pkl.dump(good_configs, open("./good_configs.pkl", "wb"))
pkl.dump(bad_configs, open("./bad_configs.pkl", "wb"))

### For T5: Window Size 1 - 7 

In [ ]:
model_name = "t5-large"
for i in range(1,7,1):
    results_window = generate(model_name, summaries[:10], True, i,1)
    print(f"Compression rate for {i}: {results_window['compression_rate']}")
for i in range(1,7,1):
    results_window = generate(model_name, summaries[:20], True, i,1)
    print(f"Compression rate for {i}: {results_window['compression_rate']}")
for i in range(1,7,1):
    results_window = generate(model_name, summaries[:30], True, i,1)
    print(f"Compression rate for {i}: {results_window['compression_rate']}")
for i in range(1,7,1):
    results_window = generate(model_name, summaries[:40], True, i,1)
    print(f"Compression rate for {i}: {results_window['compression_rate']}")
for i in range(1,7,1):
    results_window = generate(model_name, summaries[:50], True, i,1)
    print(f"Compression rate for {i}: {results_window['compression_rate']}")
print(results_window["generated summaries"][0])

### For Bart: Window-Size 1-7 

In [ ]:
model_name = "facebook/bart-large-cnn"
for i in range(1,7,1):
    results_window = generate(model_name, summaries[:10], True, i,1)
    print(f"Compression rate for {i}: {results_window['compression_rate']}")
for i in range(1,7,1):
    results_window = generate(model_name, summaries[:20], True, i,1)
    print(f"Compression rate for {i}: {results_window['compression_rate']}")
for i in range(1,7,1):
    results_window = generate(model_name, summaries[:30], True, i,1)
    print(f"Compression rate for {i}: {results_window['compression_rate']}")
for i in range(1,7,1):
    results_window = generate(model_name, summaries[:40], True, i,1)
    print(f"Compression rate for {i}: {results_window['compression_rate']}")
for i in range(1,7,1):
    results_window = generate(model_name, summaries[:50], True, i,1)
    print(f"Compression rate for {i}: {results_window['compression_rate']}")
print(results_window["generated summaries"][0])